# Imitatoin Learning 모델 2
---
## Description
> Behavioral Cloning 방식을 넘어 Data Aggregation 방식 도입.

* 실제 로봇과 상호작용 필요
* model v_1 과 다른 구조 필요
* 

## todo
* [ ] OpenAI gym like style
* [ ] 학습데이터 로컬에 파일로 save & load
* [ ] 학습에 사용된 원본(color) 데이터도 따로 저장하기
* [ ] 터틀봇과 데이터 송수신하기 - python 라이브러리 형태로 ros topic 에 접근이 가능한가? 되면 바로 python 레벨에서 통신가능
* [ ] jupyter 버전 / terminal 스크립트 버전 둘 다 돌아가게 만들기

## Issue
* jupyter 에서는 non-blocking keyboard 입력이 어려운 듯
* 그래서 그냥 스크립트로 터미널에서 돌려야 할 듯
* 그리고 linux 에서는 termios 모듈을 써야하고 windows 에서는 msvcrt 모듈을 써야함!!

## 터틀봇 컨트롤
[키보드 제어 링크](http://emanual.robotis.com/docs/en/platform/turtlebot3/teleoperation/#keyboard)

In [43]:
import platform
import cv2
import tensorflow as tf
import h5py
import numpy as np

In [2]:
os_cur = platform.system() # 현재 os 시스템 정보 # 윈도우 or 리눅스
print('> 당신의 os 는 {} 입니다.'.format(os_cur))

> 당신의 os 는 Windows 입니다.


In [84]:
class Env:
    def __init__(self, database, model, sess):
        self.database = database # 외부에 학습데이터 저장해놓기. 데이터는 소중하니까
        self.model = model # CNN 모델
        self.sess = sess # session 객체
        print('> Env 생성')
    
    def reset(self, image, ridar=None):
        # 터틀봇으로부터 카메라 데이터 및 라이다 데이터를 수신하여 반환하는 메서드
        # 동시에 초기화
        # ** 통신필요 **
        # recv 함수들 모두 인자 수정해야함!!
        observation = image
        print('> Env started!')
        return observation
    
    def pause(self):
        # 주행을 멈춘다.
        # for learning or exit
        pass
    
    def recv_data(self, image, ridar=None):
        # 터틀봇으로부터 카메라 데이터 및 라이다 데이터를 수신하여 반환하는 메서드
        # ** 통신필요 **
        observation = image
        return observation
    
    def recv_command_fromhuman(self):
        # human 이 수동으로 조종한 데이터를 수신하여 반환하는 메서드
        # 단순히 기능으로 판단
        
        key = getkey() # 키보드 입력 없을 때 return 이 None 인지???
        if(key): # key - val mapping, require nonzero
            print('- [debug] human 이 조종함. {}'.format(key))
            val = key
            return val
        else:
            return False
        
    def convert_to_command(self, key):
        command = np.array(lend(label))
        command[key] = 1
        return command
    
    def send_data(self, policy):
        # 터틀봇에 policy 즉, action command 를 보내는 메서드
        # ** 통신필요 **
        try:
            pass
        except:
            # 에러 메시지 띄우고 프로그램 종료
            pass
        
        return action        
    
    def get_dataset(self, observation, label):
        # 외부 데이터베이스에 현재상태에 대한 데이터를 제공하기위한 메서드
        return observation, label
    
    def step(self, processed):
        # 메인 루프.
        # 영상획득 - 전처리(외부로부터 진행) - 개입여부 확인 - 개입했다면 데이터 추가하고 개입한대로 action 수행
        #                                   - 아니라면 도출된 policy 결과로 action 수행
      
        self.send_data(action) # 터틀봇에 action 을 송신한다.
        
        image_original = self.recv_data(0)
        return image_original

In [102]:
class CVHelper:
    # openCV 작업만 하는 class
    def __init__(self):
        print('> CVHelper 생성')
    
    def preprocess_frame(self, frame):
        # 전처리 한 frame 을 반환해줌
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret_bin, binary = cv2.threshold(gray, 80, 255, cv2.THRESH_BINARY)
        
        if(ret_bin):
            return binary
        else:
            return None
    
    def origin_to_obs(self, video_file, label_file=None):
        # 원본 이미지파일 --> 전처리된 학습데이터
        # 를 우선 txt 파일로 저장
        cap = cv2.VideoCapture(video_file)
        width, height = cap.get(3), cap.get(4) # w, h
        tmp_data = []
        tmp_label = label_file
        print('> 학습데이터 생성 중 - {}..'.format(data_base.name))
        
        while(cap.isOpened()):
            ret, frame = cap.read()
            if(ret):
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                ret_bin, binary = cv2.threshold(gray, 80, 255, cv2.THRESH_BINARY)
                tmp_data.append(binary)
        
        data_base = h5py.File('train_data.h5', 'w') # 파일에 저장, 최초 생성
        data_base.create_dataset('image', data=tmp_data, dtype='float32')
        data_base.create_dataset('label', data=tmp_label, dtype='float32')
        data_base.close()
        
        print('> 학습데이터 생성 완료 - {}..'.format(data_base.name))

In [103]:
class CNN:
    def __init__(self, h, w, sess):
        self.size_h = h
        self.size_w = w
        self.sess = sess
        self.model = self.make_model()
    
    def make_model(self):
        # 모델
        self.observation = tf.placeholder(shape=[None, self.size_h, self.size_w, 1], dtype=tf.float32) # 이미지 데이터
        self.label = tf.placeholder(shape=[None, 3], dtype=tf.int16) # 라벨 데이터
        
        self.w_in = tf.Variable(tf.random_normal([5,5,1,8], stddev=.01)) # conv 1 가중치
        self.l1 = tf.nn.conv2d(self.observation, self.w_in, strides=[1,1,1,1], padding='SAME')
        self.l1 = tf.nn.relu(self.l1)
        self.l1 = tf.nn.max_pool(self.l1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        
        self.w_out = tf.Variable(tf.random_normal(shape=[self.size_w//2*self.size_h//2*8, 3], stddev=.01))
        self.b = tf.Variable(tf.random_normal([3]))
        self.h_flat = tf.reshape(self.l1, [-1, self.size_w//2*self.size_h//2*8])
        
        self.output = tf.matmul(self.h_flat, self.w_out) + self.b
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.output, labels=self.label))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=.001).minimize(self.cost)
        print('> 모델 생성 완료')
        
    def train(self, batch_in, batch_label):
        # 학습시행.
        _, cost = self.sess.run([self.optimizer, self.cost], feed_dict={self.observation:batch_in, self.label:batch_label})
        return cost
    
    def policy(self, input_data):
        # policy 만 도출하기
        output = self.sess.run(self.output, feed_dict={self.observation:input_data})
        return output
    
    def test(self, in_test, label_test):
        # 길을 잘 인식하는지 테스트.
        # 테스트 데이터 셋을 미리 저장해놓고 검증하는 메서드
        in_test_copied = copy.deepcopy(in_test)
        cnt = 0
        for step in range(len(in_test_copied)):
            key = cv2.waitKey(10) & 0xFF
            if(key == ord('q')):
                break
            out = self.sess.run(self.output, feed_dict={self.observation:in_test_copied[step:step+1]})
            idx = np.argmax(out, axis=1)
            one_hot = np.zeros_like(out)
            one_hot[0][idx[0]] = 1
            '''if(np.all(one_hot == label_test[])):
                cnt += 1'''
            cv2.putText(in_test_copied[step], 'command : {}'.format(one_hot[0]), (10, 30), cv2.FONT_HERSHEY_COMPLEX, .3, (0,0,255), 1)
            cv2.imshow('testing..', in_test_copied[step])
        
        cv2.destroyAllWindows()
        accuracy = cnt / len(in_test)
        print('acc :', accuracy)
    
    def test_live(self):
        # 실시간으로 길을 잘 인식하는지 테스트하는 메서드.
        out = self.sess.run(self.output, feed_dict={self.observation:d})


## 구동

In [149]:
env = Env(None,None,None)
helper = CVHelper()
helper.origin_to_obs('test.txt') # 원본으로부터 학습데이터 파일로 생성

with h5py.File('train_data.h5', 'r') as f: # binary 파일로 학습데이터 다루기
    batch_in = f['image']
    batch_label = f['label']
    print('> 사전학습 데이터 load & setting 완료')
    
width = 100
height = 100
total_epoch = 1
init = tf.global_variables_initializer()

> Env 생성
> CVHelper 생성
> 학습데이터 생성 중 - data.txt..


KeyboardInterrupt: 

In [53]:
# 사전학습하기
# 우선 터틀봇을 조종하며 얻은 영상과 라벨 데이터로 CNN 사전학습하기.
saver = tf.train.Saver()
with tf.Session() as sess:
    model = CNN(width, height, sess)
    sess.run(init)
    print('> 사전 학습 중..')
    for epoch in range(total_epoch):
        cost = model.train(batch_in, batch_label)
        print('{}/{} epoch, cost : {}'.format(epoch+1, total_epoch, cost))
        
    save_path = saver.save(sess, 'model_drive') # 모델 저장
    #saver.save(sess, "my_test_model", global_step=1000)
    print('> 학습완료, 모델의 파라미터가 {} 에 저장됨.'.format(save_path))

> 모델 생성 완료


ValueError: Cannot feed value of shape () for Tensor 'Placeholder_12:0', which has shape '(?, 100, 100, 1)'

In [8]:
# 본격 실시간 학습
# 모델 파라미터 load, 학습 데이터 load

data_base = h5py.File('data_base.h5', 'a') # 학습데이터
saver = tf.train.Saver()

with tf.Session() as sess:
    print('> 터틀봇 실시간 구동시작.')
    saver.restore(sess, save_path)
    print('> 모델의 파라미터가 로드 됨. {}'.format(save_path))
    observation = env.reset(0) # env 초기화
    images_to_add = []
    labels_to_add = []
    print('> running...')
    
    while(False):
        processed_image = helper.preprocess_frame(observation)
        key = env.recv_command_fromhuman()

        if(key):
            action = env.convert_to_command(key) # command --> action
            images_to_add.append(processed_image) # input 데이터 추가
            labels_to_add.appedn(action) # label 데이터 추가
        else:
            policy = model.policy(processed_image)
            print('[debug] policy shape : {}'.format(policy))
            action = np.array(len(label))
            argmax = np.argmax(policy) # CNN --> policy --> action
            action[argmax] = 1

        observation = env.step(action)
        
        # 키보드로 종료 커맨드 내리면 일단 종료하고
        # 터틀봇 끄고 / 학습 / 파라미터 저장
        if(getkey()):
            print('> 터틀봇 구동 종료.')
            # 데이터 추가
            print('> data aggregating...')
            base_image_data = data_base['image']
            base_label_data = data_base['label']
            new_image_data = base_image_data + images_to_add
            new_label_data = base_label_data + labels_to_add
            
            data_base['image'] = new_image_data
            data_base['label'] = new_label_data
            
            # 학습할 데이터 할당
            batch_in = new_image_data
            batch_label = new_label_data
            
            # 학습
            total_epoch = 1
            print('> 재학습 중...')
            for epoch in range(total_epoch):
                model.train(batch_in, batch_label)
                print('{}/{} epoch, cost : {}'.format(epoch+1, total_epoch, cost))
                
            saver.save(sess, save_path)
            print('> 학습완료, 모델의 파라미터가 {} 에 저장'.format(save_path))
            break

data_base.close()

NameError: name 'observation' is not defined